EfficientNetで失敗
Unable to find out a correct type for tensor type = 20 of normalization/Reshape_1/ReadVariableOp/resource:0


VGGでFunctionalAPIではなくSequentialなやつでやってみる

## 犬と猫を分類するモデルを構築

### 1. 画像の読み込み

まずは簡単に画像の読み込みの方法を紹介します。

追加

In [1]:
import sys
import pprint
pprint.pprint(sys.path)

['/Users/takekazukitagishi/Desktop/情報学館授業資料/s２/特論７/univ_tokyo_2020/tensorflow',
 '/Applications/anaconda3/lib/python37.zip',
 '/Applications/anaconda3/lib/python3.7',
 '/Applications/anaconda3/lib/python3.7/lib-dynload',
 '',
 '/Applications/anaconda3/lib/python3.7/site-packages',
 '/Applications/anaconda3/lib/python3.7/site-packages/aeosa',
 '/Applications/anaconda3/lib/python3.7/site-packages/IPython/extensions',
 '/Users/takekazukitagishi/.ipython']


In [2]:
sys.path.insert(1,'/Users/takekazukitagishi/Desktop/情報学館授業資料/s２/特論７/univ_tokyo_2020/tensorflow/venv/lib/python3.7/site-packages')

In [3]:
import sys
import pprint
pprint.pprint(sys.path)

['/Users/takekazukitagishi/Desktop/情報学館授業資料/s２/特論７/univ_tokyo_2020/tensorflow',
 '/Users/takekazukitagishi/Desktop/情報学館授業資料/s２/特論７/univ_tokyo_2020/tensorflow/venv/lib/python3.7/site-packages',
 '/Applications/anaconda3/lib/python37.zip',
 '/Applications/anaconda3/lib/python3.7',
 '/Applications/anaconda3/lib/python3.7/lib-dynload',
 '',
 '/Applications/anaconda3/lib/python3.7/site-packages',
 '/Applications/anaconda3/lib/python3.7/site-packages/aeosa',
 '/Applications/anaconda3/lib/python3.7/site-packages/IPython/extensions',
 '/Users/takekazukitagishi/.ipython']


In [4]:
import onnx
import keras2onnx

追加終わり

In [5]:
import warnings
warnings.filterwarnings('ignore')

`train` のフォルダに入っている画像を読み込みます。
`glob` を利用すると簡単にファイルを検索できます。

In [6]:
from glob import glob

In [7]:
dog_filepaths = glob('train/dog/*.jpg')
cat_filepaths = glob('train/cat/*.jpg')

In [8]:
from PIL import Image

上記の通り、Pillow 独自の方を持っているため、計算で扱う場合には numpy の ndarray に変換しておきましょう。

In [9]:
import numpy as np

### 2. クラスラベルの割り振り

画像の読み込み方がわかったため、クラスのラベルを割り振っていきましょう。

In [10]:
# 入力値:x, 目標値: t
x, t = [], []

In [11]:
# 犬
for filepath in dog_filepaths:
    img = Image.open(filepath)
    img = img.resize((224, 224))#追加
    img = np.array(img)
    x.append(img)
    t.append(np.array(0))  # 犬は 0 とする

In [12]:
# 猫
for filepath in cat_filepaths:
    img = Image.open(filepath)
    img = img.resize((224, 224))#追加
    img = np.array(img)
    x.append(img)
    t.append(np.array(1))  # 猫は 0 とする

In [13]:
# 格納された枚数
len(x), len(t)

(300, 300)

In [14]:
# 全体を numpy の形式に変換
x = np.array(x)  # f は float32
t = np.array(t)  # i は int32

In [15]:
# 300(枚), 224(height), 224(width), 3(channels)
x.shape

(300, 224, 224, 3)

In [16]:
t.shape

(300,)

x の値の範囲を正規化しておくと学習が効率的に進むことが多いため、こちらも行なっておきましょう。学習係数などを調整する場合など、必ず必要ではありません。

In [17]:
# 特徴量の正規化 (0~1の範囲に)
x = x / 255

### 3. 訓練データと検証データに分割

tensorflow では訓練データ (train) と検証データ (val) を分ける機能がないため、scikit-learn の `train_test_split` を利用します。

なお、テストデータ (test) は私の手元にしかないため、手持ちのデータの中で以下にテストデータを模擬的に表現できるかのために検証データがあります。精度を高めたいときには**交差検証法 (cross validation)** も選択肢としてあるため、こちらも必要であれば挑戦してみましょう。

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
# train : val = 0.7 : 0.3 の割合で分割する
train_x, val_x, train_t, val_t = train_test_split(x, t, train_size=0.7, random_state=0)

In [20]:
# サイズの確認（訓練データ）
train_x.shape, train_t.shape

((210, 224, 224, 3), (210,))

In [21]:
# サイズの確認（検証データ）
val_x.shape, val_t.shape

((90, 224, 224, 3), (90,))

### 4. モデルを定義

今回は非常にシンプルな構成の CNN でモデルを構築します。あまり良い精度が出ないようにしてあるため、ここのモデルの構造を工夫して精度を高めましょう。

ヒント
- VGG, ResNet, MobileNet
- ファインチューニング

複雑なモデルを構築する場合には GPU がないと遅い場合もあるため、[こちらの手順](https://www.kikagaku.ai/tutorial/guide_for_beginners/learn/platform_environment) を参考に Google Colab を使ってみると良いでしょう。

In [22]:
import tensorflow as tf

モデル構築の際に乱数のシードも固定しておきましょう。

In [23]:
import os
import random

def reset_seed(seed=0):

    os.environ['PYTHONHASHSEED'] = '0'
    random.seed(seed) #　random関数のシードを固定
    np.random.seed(seed) #numpyのシードを固定
    tf.random.set_seed(seed) #tensorflowのシードを固定

In [24]:
# シードの固定を実行
reset_seed()

In [25]:
from tensorflow.keras import models, layers

以下のようなモデルを定義していきましょう。

- Original: (224, 224, 3)
- => Convolution (Relu) => (224, 224, 6)
- => Half Pooling => (112, 112, 6)
- => ベクトル化 (112×112×6)
- => 全結合層 (112×112×6 => 100), Relu
- => 全結合層 (100 => 2), Softmax

### モデル

VGGやってみる

In [26]:

epochs = 5
batch_size = 16
input_shape = (224, 224, 3)
class_count = 2


In [27]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD

In [28]:
# モデルの定義
model = models.Sequential([
    # Convolution
    layers.Conv2D(64, (3, 3), activation='relu', input_shape=input_shape),
    layers.Conv2D(64, (3, 3), activation='relu'),
    # Pooling
    layers.MaxPooling2D(pool_size=(2, 2)),
    #Dropout
    layers.Dropout(0.25),
    #Convolution
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.Conv2D(128, (3, 3), activation='relu'),
    #Pooling
    layers.MaxPooling2D(pool_size=(2, 2)),
    # Convolution
    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.Conv2D(256, (3, 3), activation='relu'),
    # Pooling
    layers.MaxPooling2D(pool_size=(2, 2)),
    # Convolution
    layers.Conv2D(512, (3, 3), activation='relu'),
    layers.Conv2D(512, (3, 3), activation='relu'),
    layers.Conv2D(512, (3, 3), activation='relu'),
    # Pooling
    layers.MaxPooling2D(pool_size=(2, 2)),

    # Convolution
    layers.Conv2D(512, (3, 3), activation='relu'),
    layers.Conv2D(512, (3, 3), activation='relu'),
    layers.Conv2D(512, (3, 3), activation='relu'),
    # Pooling
    layers.MaxPooling2D(pool_size=(2, 2)),

    # ベクトル化 (Flatten)
    layers.Flatten(),
    # 全結合層
    layers.Dense(256, activation='relu'),

    # 全結合層
    layers.Dense(1, activation='sigmoid') 
])

### コンパイル

In [29]:

model.compile(
    optimizer='rmsprop',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


### ヒストリー

In [30]:

# モデルの学習
history = model.fit(
    train_x, train_t,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(val_x, val_t)
)


Epoch 1/5
14/14 [==============================] - 69s 5s/step - loss: 165.1337 - accuracy: 0.4667 - val_loss: 0.6936 - val_accuracy: 0.4667
Epoch 2/5
14/14 [==============================] - 78s 6s/step - loss: 0.6928 - accuracy: 0.5238 - val_loss: 0.7403 - val_accuracy: 0.4667
Epoch 3/5
14/14 [==============================] - 67s 5s/step - loss: 0.6979 - accuracy: 0.5143 - val_loss: 0.6910 - val_accuracy: 0.5333
Epoch 4/5
14/14 [==============================] - 63s 5s/step - loss: 0.6941 - accuracy: 0.4762 - val_loss: 0.6944 - val_accuracy: 0.4667
Epoch 5/5
14/14 [==============================] - 63s 4s/step - loss: 0.6936 - accuracy: 0.5143 - val_loss: 0.6952 - val_accuracy: 0.4667


追加終わり

### 5. モデルの学習

Tenrsoflow では定義したモデルをコンパイルする必要がありました。最適化のアルゴリズムを確率的勾配降下法 (SGD) として選択します。

ここのアルゴリズムの選択と学習係数の設定も精度向上のポイントですので、[こちらの記事](https://www.tensorflow.org/guide/keras/train_and_evaluate) などを参考にしながら進めてみましょう。

train に関して accuracy が 0.76, validation に関しては accuracy が 0.49 となっており、**過学習 (overfitting)** が生じていることがわかります。

この辺りも踏まえて、予測の精度を上げられるように色々な手法に挑戦してみましょう。比較的簡単なところでは以下を調整してみると良いでしょう。

- モデルの構成（VGG, ResNet, MobileNet など）
- ファイチューニング
- 最適化アルゴリズムの選択
- 学習係数

### 5. モデルの保存

推論にて使用するために学習済みモデルをファイルに出力しましょう。

In [31]:
# HDF5 という形式で保存（TensorFlowではこちらを用いるようです）
model.save('dog_cat_cnn.h5')

これで作業フォルダの中に `dog_cat_cnn.h5` というファイルが出力できていれば成功です。

### 7. ONNX形式で出力

ONNX は TensorFlow や PyTorch 問わず、モデルの形式を標準化するプロジェクトです。  
最近では、こちらの形式に統一しておくほうが汎用性が高くなっているため、推論サーバーでは ONNX 形式を使用します。

onnxruntime では通常よりも速度が速いと言われています。

In [32]:
import onnxruntime
onnxruntime.__version__

'1.4.0'

In [33]:
import onnx
import keras2onnx
#keras2onnx.__version__

In [34]:
# Keras -> ONNX の変換
onnx_model = keras2onnx.convert_keras(model, model.name)

tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 84 -> 40


In [35]:
# モデルを保存
keras2onnx.save_model(onnx_model, 'dog_cat_cnn.onnx')

In [36]:
# モデルの読み込み
session = onnxruntime.InferenceSession('dog_cat_cnn.onnx')

In [37]:
# 推論に必要な構造の抽出
session.get_modelmeta()
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

In [38]:
input_name, output_name

('conv2d_input', 'dense_1')

In [39]:
# 推論
y_probs = session.run([output_name], {input_name: x})[0]
y_probs

InvalidArgument: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Unexpected input data type. Actual: (N11onnxruntime17PrimitiveDataTypeIdEE) , expected: (N11onnxruntime17PrimitiveDataTypeIfEE)

In [ ]:
# ラベル
y = np.argmax(y_prob)
y